#### 1. Preparar el entorno

In [ ]:
#instalar biopython
#!pip3 install biopython

In [8]:
#Cargar librerías
from Bio import SeqIO
import pandas as pd
from Bio import Entrez

In [9]:
#acceso a entrez
Entrez.email = "hoaxaca@ccg.unam.mx"  # IMPORTANTE!!!
# handle con einfo
handle = Entrez.einfo()
result = handle.read() 
handle.close()
#chequemos qué hay en einfo
print(result)

b'<?xml version="1.0" encoding="UTF-8" ?>\n<!DOCTYPE eInfoResult PUBLIC "-//NLM//DTD einfo 20190110//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20190110/einfo.dtd">\n<eInfoResult>\n<DbList>\n\n\t<DbName>pubmed</DbName>\n\t<DbName>protein</DbName>\n\t<DbName>nuccore</DbName>\n\t<DbName>ipg</DbName>\n\t<DbName>nucleotide</DbName>\n\t<DbName>structure</DbName>\n\t<DbName>genome</DbName>\n\t<DbName>annotinfo</DbName>\n\t<DbName>assembly</DbName>\n\t<DbName>bioproject</DbName>\n\t<DbName>biosample</DbName>\n\t<DbName>blastdbinfo</DbName>\n\t<DbName>books</DbName>\n\t<DbName>cdd</DbName>\n\t<DbName>clinvar</DbName>\n\t<DbName>gap</DbName>\n\t<DbName>gapplus</DbName>\n\t<DbName>grasp</DbName>\n\t<DbName>dbvar</DbName>\n\t<DbName>gene</DbName>\n\t<DbName>gds</DbName>\n\t<DbName>geoprofiles</DbName>\n\t<DbName>homologene</DbName>\n\t<DbName>medgen</DbName>\n\t<DbName>mesh</DbName>\n\t<DbName>nlmcatalog</DbName>\n\t<DbName>omim</DbName>\n\t<DbName>orgtrack</DbName>\n\t<DbName>pmc</DbName>\n

#### 2. Obtener genomas en formato GenBank

In [48]:
#Leer DataFrame con información de los genomas de interés
genomes = pd.read_csv("data/PRJNA842432_AssemblyDetails.tsv", sep = "\t")
genomes 

,Assembly,Level,WGS,BioSample,Strain,Taxonomy
0,GCA_024199795.1,Contig,JANADK010000001.1,SAMN28667590,CCGB01,Bradyrhizobium sp. CCGB01
1,GCA_024199845.1,Contig,JANADO000000000.1,SAMN28667586,CCGB12,Bradyrhizobium sp. CCGB12
2,GCA_024199855.1,Contig,JANADN000000000.1,SAMN28667587,CCGB20,Bradyrhizobium sp. CCGB20
3,GCA_024199935.1,Contig,JANADS000000000.1,SAMN28667582,CCGUVB14,Bradyrhizobium sp. CCGUVB14
4,GCA_024199925.1,Scaffold,JANADR000000000.1,SAMN28667583,CCGUVB1N3,Bradyrhizobium sp. CCGUVB1N3
5,GCA_024199885.1,Scaffold,JANADQ000000000.1,SAMN28667584,CCGUVB23,Bradyrhizobium sp. CCGUVB23
6,GCA_024199945.1,Contig,JANADP000000000.1,SAMN28667585,CCGUVB4N,Bradyrhizobium sp. CCGUVB4N
7,GCA_000296215.2,Contig,CP013949.1,SAMN02469422,CCGE-LA001,Bradyrhizobium sp. CCGE-LA001


In [49]:
#Obtener los GenBank de los genomas de interés
for i, accession in enumerate(genomes["WGS"]):
    strain = genomes["Strain"][i]
    out = "data/gbk_ncbi" + strain + ".gbk"
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    SeqIO.write(record, out, "genbank")
    print("Archivo GenBank generado para la cepa:", strain)


Archivo GenBank generado para la cepa: CCGB01
Archivo GenBank generado para la cepa: CCGB12
Archivo GenBank generado para la cepa: CCGB20
Archivo GenBank generado para la cepa: CCGUVB14
Archivo GenBank generado para la cepa: CCGUVB1N3
Archivo GenBank generado para la cepa: CCGUVB23
Archivo GenBank generado para la cepa: CCGUVB4N
Archivo GenBank generado para la cepa: CCGE-LA001


#### 3. Cargar los gbk al software en linea [IslandViewer4](https://www.pathogenomics.sfu.ca/islandviewer), detectar las islas simbióticas y definir coordenadas extendidas en un DataFrame 

#### 4. Extraer la región correspondiente a la isla simbiótica de cada cepa

In [50]:
#Cargar el DataFrame con las coordenadas de las islas simbióticas
coordinates = pd.read_csv("data/coordinates_islands.tsv", sep="\t")
coordinates

,strain,start,end
0,CCGUVB1N3,7234698,8205292
1,CCGUVB23,1653368,2782328
2,CCGUVB14,1205589,2378822
3,CCGUVB4N,1088213,2225466
4,CCGB20,1472672,1924306
5,CCGB01,1591475,2046468
6,CCGB12,5766140,6480540
7,CCGE-LA001,6125803,6929059


In [51]:
# Definir función para extraer región
def extraer_region_gbk(gbk_file, start, end, output_file):
    record = SeqIO.read(gbk_file, "genbank")
    region = record[start-1:end]    
    # Modificar los nombres de los CDS e incluir la posición
    for feature in region.features:
        if feature.type == "CDS":
            feature.qualifiers["locus_tag"][0] += f"_{feature.location.start.position}_{feature.location.end.position}"
    with open(output_file, "w") as out:
        SeqIO.write(region, out, "genbank")

In [66]:
#Extraer las regiones de cada cepa
for s, names in enumerate(coordinates['strain']):
    strain = coordinates['strain'][s]
    input_file = "data/gbk_island_viewer/" + strain + ".gbk"
    start_position = coordinates['start'][s]
    end_position = coordinates['end'][s]
    output_file = "results/" + strain + "_SI.gbk"
    extraer_region_gbk(input_file, start_position, end_position, output_file)
    print("Archivo GenBank extraido de la cepa:", strain)

Archivo GenBank extraido de la cepa: CCGUVB1N3
Archivo GenBank extraido de la cepa: CCGUVB23
Archivo GenBank extraido de la cepa: CCGUVB14
Archivo GenBank extraido de la cepa: CCGUVB4N
Archivo GenBank extraido de la cepa: CCGB20
Archivo GenBank extraido de la cepa: CCGB01


/Users/diana/opt/anaconda3/lib/python3.9/site-packages/Bio/GenBank/__init__.py:751: BiopythonParserWarning: Non-upper case molecule type in LOCUS line: dna
  warnings.warn(


Archivo GenBank extraido de la cepa: CCGB12
Archivo GenBank extraido de la cepa: CCGE-LA001


#### 5. Comparar las islas simbióticas de cada cepa

###### Para esta parte se usó bash en la terminal
conda activate clinker
clinker results/*.gbk -p Bradys_IS.html -mu results/Bradys_IS.matrix

In [67]:
6.

'/Users/diana/Dropbox/Bradys_islands_integrases'